In [1]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate

from the_bootstrap_approach.conditions import FullThrottleConditions

from the_bootstrap_approach.equations import *
from the_bootstrap_approach.mixture import Mixture
from the_bootstrap_approach.performance import (
    best_rate_of_climb,
    best_range_and_optimum_cruise,
)

from n51sw_dataplate import N51SW


def performance_profiles(gross_aircraft_weight):
    best_rate_of_climb_profile = []
    best_range_profile = []
    optimum_cruise_profile = []

    # Piper publishes a service ceiling of 17,500 ft. Originally, I only wanted to
    # determine performance up to 18,000 ft, as our Cannula's are only approved that
    # high. However, I reached 18,000 ft, the plane wanted to keep climbing, and I
    # had a mask. I was able to reach 22,500 ft in the Dakota, and the performance I
    # saw is identical to these charts.
    for pressure_altitude in np.arange(0, 26000, 1000):
        oat_f = british_standard_temperature(pressure_altitude)

        best_rate = best_rate_of_climb(
            N51SW,
            FullThrottleConditions(
                N51SW,
                gross_aircraft_weight,
                pressure_altitude,
                oat_f,
                Mixture.BEST_POWER,
                2400,
            ),
        )

        # If the rate of climb is negative, the aircraft isn't able to sustain level flight.
        if best_rate[4] > 0:
            best_rate_of_climb_profile.append(
                np.insert(
                    best_rate,
                    0,
                    pressure_altitude,
                )
            )

        best_range, optimum_cruise = best_range_and_optimum_cruise(
            N51SW, gross_aircraft_weight, pressure_altitude, oat_f, Mixture.BEST_ECONOMY
        )

        if best_range[4] > 0:
            best_range_profile.append(np.insert(best_range, 0, pressure_altitude))
        if optimum_cruise[4] > 0:
            optimum_cruise_profile.append(
                np.insert(optimum_cruise, 0, pressure_altitude)
            )

    return best_rate_of_climb_profile, best_range_profile, optimum_cruise_profile


best_rate_of_climb_profiles = {}
best_range_profiles = {}
optimum_cruise_profiles = {}

for gross_aircraft_weight in (2000, 2500, 2750, 3000):
    (
        best_rate_of_climb_profile,
        best_range_profile,
        optimum_cruise_profile,
    ) = performance_profiles(gross_aircraft_weight)

    best_rate_of_climb_profiles[gross_aircraft_weight] = best_rate_of_climb_profile
    best_range_profiles[gross_aircraft_weight] = best_range_profile
    optimum_cruise_profiles[gross_aircraft_weight] = optimum_cruise_profile


headers = ("h_ρ", "KCAS", "KTAS", "η", "h", "γ", "RPM", "% bhp", "gph", "mpg")
tablefmt = "latex_booktabs"
floatfmt = (".0f", ".1f", ".1f", ".4f", ".0f", ".0f", ".0f", ".2f", ".2f", ".2f")

for gross_aircraft_weight, profile in best_rate_of_climb_profiles.items():
    print(
        f"Best Rate of Climb (V_y), {N51SW.configuration}, Full Throttle, 2400 RPM, {Mixture.BEST_POWER.value}",
        "\n",
        f"W = {gross_aircraft_weight}, ISA+0, V_w = 0",
        "\n",
        tabulate(
            np.delete(profile, [9], axis=1),
            headers=headers,
            tablefmt=tablefmt,
            floatfmt=floatfmt,
        ),
        "\n",
        sep="",
    )

for gross_aircraft_weight, profile in best_range_profiles.items():
    print(
        f"Best Range, {N51SW.configuration}, {Mixture.BEST_ECONOMY.value}",
        "\n",
        f"W = {gross_aircraft_weight}, ISA+0, V_w = 0",
        "\n",
        tabulate(
            np.delete(profile, [9], axis=1),
            headers=headers,
            tablefmt=tablefmt,
            floatfmt=floatfmt,
        ),
        "\n",
        sep="",
    )

for gross_aircraft_weight, profile in optimum_cruise_profiles.items():
    print(
        f"Optimum Cruise (Carson's Speed), {N51SW.configuration}, {Mixture.BEST_ECONOMY.value}",
        "\n",
        f"W = {gross_aircraft_weight}, ISA+0, V_w = 0",
        "\n",
        tabulate(
            np.delete(profile, [9], axis=1),
            headers=headers,
            tablefmt=tablefmt,
            floatfmt=floatfmt,
        ),
        "\n",
        sep="",
    )

Best Rate of Climb (V_y), Flaps Up, Full Throttle, 2400 RPM, Best Power
W = 2000, ISA+0, V_w = 0
\begin{tabular}{rrrrrrrrrr}
\toprule
   h\_ρ &   KCAS &   KTAS &      η &    h &    γ &   RPM &   \% bhp &   gph &   mpg \\
\midrule
     0 &   79.8 &   79.8 & 0.6653 & 1769 & 1330 &  2400 &  100.00 & 18.02 &  4.43 \\
  1000 &   78.8 &   80.0 & 0.6669 & 1694 & 1271 &  2400 &   96.71 & 17.42 &  4.59 \\
  2000 &   77.7 &   80.0 & 0.6681 & 1621 & 1215 &  2400 &   93.50 & 16.85 &  4.75 \\
  3000 &   76.7 &   80.2 & 0.6697 & 1549 & 1159 &  2400 &   90.35 & 16.28 &  4.93 \\
  4000 &   75.7 &   80.3 & 0.6714 & 1477 & 1104 &  2400 &   87.28 & 15.73 &  5.11 \\
  5000 &   74.7 &   80.5 & 0.6730 & 1407 & 1049 &  2400 &   84.28 & 15.18 &  5.30 \\
  6000 &   73.7 &   80.6 & 0.6747 & 1338 &  996 &  2400 &   81.35 & 14.66 &  5.50 \\
  7000 &   72.8 &   80.9 & 0.6768 & 1270 &  942 &  2400 &   78.48 & 14.14 &  5.72 \\
  8000 &   71.8 &   81.0 & 0.6785 & 1203 &  891 &  2400 &   75.68 & 13.64 &  5.94 \\
  900